In [ ]:
import pandas as pd

import os

file_path = './data.xlsx'
output_folder = './matrices_por_dia'

df = pd.read_excel(file_path, sheet_name='Hoja1', skiprows=2)

df.columns = ['Index', 'Fecha', 'Nro.Estacion', 'Estacion'] + [f"{i:02}" for i in range(24)] + ['Total']
df = df[['Fecha', 'Nro.Estacion'] + [f"{i:02}" for i in range(24)]]

df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df = df.dropna(subset=['Fecha', 'Nro.Estacion'])

df.fillna(0, inplace=True)

os.makedirs(output_folder, exist_ok=True)

for date, group in df.groupby(df['Fecha'].dt.date):
    matrix = group.set_index('Nro.Estacion')[[f"{i:02}" for i in range(24)]].T
    matrix = matrix.astype(int)  # Convertir a valores enteros
    file_path = os.path.join(output_folder, f"matriz_{date}.csv")
    matrix.to_csv(file_path)

output_folder


'./matrices_por_dia'

In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import joblib  

input_folder = './matrices_por_dia'
output_folder_normalized = './matrices_normalizadas'

os.makedirs(output_folder_normalized, exist_ok=True)

scaler = MinMaxScaler()

all_data = []
metadata = []  

file_names = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
for file_name in file_names:
    file_path = os.path.join(input_folder, file_name)
    df = pd.read_csv(file_path, index_col=0) 
    all_data.append(df.values)  
    metadata.append((df.index, df.columns))  

all_data_combined = pd.concat([pd.DataFrame(data) for data in all_data]).values

scaler.fit(all_data_combined)
normalized_data_combined = scaler.transform(all_data_combined)

scaler_path = "./scaler.pkl"
joblib.dump(scaler, scaler_path)

start_idx = 0
for file_name, (original_data, (index, columns)) in zip(file_names, zip(all_data, metadata)):
    rows = original_data.shape[0]
    normalized_data = normalized_data_combined[start_idx:start_idx + rows]
    start_idx += rows

    normalized_df = pd.DataFrame(normalized_data, index=index, columns=columns)
    
    output_file_path = os.path.join(output_folder_normalized, file_name)
    normalized_df.to_csv(output_file_path)

print("Matrices normalizadas creadas en la carpeta:", output_folder_normalized)
print("Scaler guardado en:", scaler_path)

Matrices normalizadas creadas en la carpeta: ./matrices_normalizadas
Scaler guardado en: ./scaler.pkl


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

input_folder = './matrices_normalizadas'
output_folder_heatmaps = './heatmaps'
output_folder_heatmaps_no_labels = './heatmaps_no_labels'

os.makedirs(output_folder_heatmaps, exist_ok=True)
os.makedirs(output_folder_heatmaps_no_labels, exist_ok=True)

file_names = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

for file_name in file_names:
    file_path = os.path.join(input_folder, file_name)
    df = pd.read_csv(file_path, index_col=0)

    plt.figure(figsize=(10, 8))
    plt.imshow(df, aspect='auto', cmap='viridis')  # Cambiar paleta si se desea
    plt.colorbar(label='Escala Normalizada')
    plt.title(f"Heatmap: {file_name}")
    plt.xlabel('Estaciones')
    plt.ylabel('Periodos de Tiempo')
    output_path = os.path.join(output_folder_heatmaps, file_name.replace('.csv', '.png'))
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

    plt.figure(figsize=(10, 8))
    plt.imshow(df, aspect='auto', cmap='viridis')  # Cambiar paleta si se desea
    plt.axis('off')  # Desactiva las etiquetas de ejes y la barra de colores
    output_path_no_labels = os.path.join(output_folder_heatmaps_no_labels, file_name.replace('.csv', '.png'))
    plt.savefig(output_path_no_labels, dpi=300, bbox_inches='tight', pad_inches=0)  # Elimina bordes blancos
    plt.close()

print("Heatmaps generados con etiquetas en la carpeta:", output_folder_heatmaps)
print("Heatmaps generados sin etiquetas en la carpeta:", output_folder_heatmaps_no_labels)


Heatmaps generados con etiquetas en la carpeta: ./heatmaps
Heatmaps generados sin etiquetas en la carpeta: ./heatmaps_no_labels
